# Materia: Aprendizaje Automático 1
# FINAL  - Parte Práctica

# Consigna:  
    
Dado el DataSet Social_Network_Missing_Values.csv **separado por punto y coma (;)** que se encuentra en la carpeta data se desea crear un modelo de **REGRESIÓN LOGÍSTICA** para pronosticar, en este caso **no se hará selección de modelo**. 

Para concretar su tarea se deberán realizar 4 etapas:  

- Preprocesamiento: limpieza de missing values
- Preprocesamiento: normalización / estandarización según se indique.
- Evaluación del modelo de Regresión Logística con ACURRACY para determinar cómo funcionará frente a observaciones no vistas.
- Crear el modelo para producción.


Observaciones 

- Cuando sea necesario randomizar usar semilla = 123, para dividir dataset usar proporción 80/20.

- Las variables son :  

- Gender: género
- Age: edad
- EstmatedSalary: salario (anual) estimado
- Purchased: ésta es la variable que queremos pronosticar. Como asume sólo 2 valores es un problema de Clasificación,

**Primera Parte**

Preprocesamiento:  

- Divida en X_train, X_test, y_train, y_test usando (80/20) y semilla 123

Limpieza de *Missing Values*

- imputar la moda en Gender
- imputar la media en Age
- imputar la mediana en EstimatedSalary

Utilice los nombres: X_train_limpio y X_test_limpio para los datasets sin nulos.




**Segunda Parte**

- En este caso no hay variables ordinales, así que deberá:  

- transformar en dummy todas las variables categóricas.  

- **Normalizar** Age

- ** Estandarizar** EstimatedSalary

Guarde lo archivos con los nombres X_train_transformado.csv y X_test_transformado.csv

**Tercer Parte**

Evaluación del Modelo.

- Calcule el ACCURACY que esperamos que tenga el modelo cuando pronostique observaciones no vistas. 

**Cuarta parte**

- Genere el modelo para producción llámelo modelo_prod  
- Obtenga los parámetros del modelo de producción



In [46]:
import pandas as pd  
import numpy as np  
from sklearn.impute import SimpleImputer   
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [47]:
df = pd.read_csv('Social_Network_Missing_Values.csv',sep=';')
df

,Gender,Age,EstimatedSalary,Purchased
0,Male,19.0,19000.0,0
1,NaN,35.0,20000.0,0
2,Female,26.0,43000.0,0
3,Female,NaN,57000.0,0
4,Male,19.0,NaN,0
...,...,...,...,...
395,Female,46.0,41000.0,1
396,Male,51.0,23000.0,1
397,Female,50.0,20000.0,1
398,Male,36.0,33000.0,0


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           385 non-null    object 
 1   Age              385 non-null    float64
 2   EstimatedSalary  385 non-null    float64
 3   Purchased        400 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 12.6+ KB


In [49]:
df= df.dropna()
df

,Gender,Age,EstimatedSalary,Purchased
0,Male,19.0,19000.0,0
2,Female,26.0,43000.0,0
5,Male,27.0,58000.0,0
6,Female,27.0,84000.0,0
8,Male,25.0,33000.0,0
...,...,...,...,...
395,Female,46.0,41000.0,1
396,Male,51.0,23000.0,1
397,Female,50.0,20000.0,1
398,Male,36.0,33000.0,0


In [50]:
x=df.drop(axis=1,columns='Purchased')
y=df['Purchased']   

In [51]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=123) 

In [52]:
columns_num=X_train.select_dtypes(np.number).columns
columns_cat=X_train.select_dtypes(include=['object','bool']).columns
t_norm=("normalizador",MinMaxScaler(feature_range=(0, 1)),['Age'])
t_estandard=("estandarizador",StandardScaler(),['EstimatedSalary'])

In [53]:
t1 = ('imputador_num',SimpleImputer(strategy="median"),['EstimatedSalary'])
t2 = ('imputador_cat',SimpleImputer(strategy="most_frequent"),['Gender'])
t3 = ('imputador_num',SimpleImputer(strategy="mean"),["Age"])

In [74]:
transformador_columnas = ColumnTransformer(transformers=[t1,t2],remainder='passthrough')

In [75]:
transformador_columnas.fit(X_train)
Train_transformado = transformador_columnas.transform(X_train)
Test_transformado=transformador_columnas.transform(X_test)

In [73]:
cols_orden=columns_num.tolist() + columns_cat.tolist()
X_train_limpio=pd.DataFrame(Train_transformado, columns=cols_orden) 
X_test_limpio=pd.DataFrame(Test_transformado, columns=cols_orden)
                                

display(X_train_limpio)
display(X_test_limpio)

,Age,EstimatedSalary,Gender
0,117000.0,Male,46.0
1,117000.0,Female,32.0
2,88000.0,Male,46.0
3,87000.0,Male,30.0
4,83000.0,Male,60.0
...,...,...,...
279,71000.0,Female,39.0
280,18000.0,Male,31.0
281,61000.0,Male,40.0
282,17000.0,Male,30.0


,Age,EstimatedSalary,Gender
0,43000.0,Male,29.0
1,23000.0,Female,20.0
2,28000.0,Male,46.0
3,25000.0,Female,34.0
4,117000.0,Female,32.0
...,...,...,...
66,72000.0,Male,37.0
67,91000.0,Male,35.0
68,139000.0,Female,44.0
69,43000.0,Male,34.0


In [57]:
X_train_limpio.columns = ['EstimatedSalary','Gender','Age']
X_test_limpio.columns = ['EstimatedSalary','Gender','Age']
display(X_train_limpio)
display(X_test_limpio)

,EstimatedSalary,Gender,Age
0,117000.0,Male,46.0
1,117000.0,Female,32.0
2,88000.0,Male,46.0
3,87000.0,Male,30.0
4,83000.0,Male,60.0
...,...,...,...
279,71000.0,Female,39.0
280,18000.0,Male,31.0
281,61000.0,Male,40.0
282,17000.0,Male,30.0


,EstimatedSalary,Gender,Age
0,43000.0,Male,29.0
1,23000.0,Female,20.0
2,28000.0,Male,46.0
3,25000.0,Female,34.0
4,117000.0,Female,32.0
...,...,...,...
66,72000.0,Male,37.0
67,91000.0,Male,35.0
68,139000.0,Female,44.0
69,43000.0,Male,34.0


In [58]:
X_todos=X_train_limpio.append(X_test_limpio,ignore_index=True)
X_todos

,EstimatedSalary,Gender,Age
0,117000.0,Male,46.0
1,117000.0,Female,32.0
2,88000.0,Male,46.0
3,87000.0,Male,30.0
4,83000.0,Male,60.0
...,...,...,...
350,72000.0,Male,37.0
351,91000.0,Male,35.0
352,139000.0,Female,44.0
353,43000.0,Male,34.0


In [59]:
tdummy=("onehot",OneHotEncoder(sparse=False), ['Gender'])


In [60]:
df_dummy_1=pd.get_dummies(X_train_limpio,columns=['Gender'])
df_dummy_1

,EstimatedSalary,Age,Gender_Female,Gender_Male
0,117000.0,46.0,0,1
1,117000.0,32.0,1,0
2,88000.0,46.0,0,1
3,87000.0,30.0,0,1
4,83000.0,60.0,0,1
...,...,...,...,...
279,71000.0,39.0,1,0
280,18000.0,31.0,0,1
281,61000.0,40.0,0,1
282,17000.0,30.0,0,1


In [61]:
transformador_columnas_dummy3= ColumnTransformer(transformers=[tdummy],remainder='passthrough')

transformador_columnas_dummy3.fit(X_todos)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(sparse=False),
                                 ['Gender'])])

In [62]:
X_train_dummy=transformador_columnas_dummy3.transform(X_train_limpio)

X_test_dummy=transformador_columnas_dummy3.transform(X_test_limpio)


In [63]:
display(X_train_dummy)
display(X_test_dummy)

array([[0.0, 1.0, 117000.0, 46.0],
       [1.0, 0.0, 117000.0, 32.0],
       [0.0, 1.0, 88000.0, 46.0],
       ...,
       [0.0, 1.0, 61000.0, 40.0],
       [0.0, 1.0, 17000.0, 30.0],
       [0.0, 1.0, 104000.0, 42.0]], dtype=object)

array([[0.0, 1.0, 43000.0, 29.0],
       [1.0, 0.0, 23000.0, 20.0],
       [0.0, 1.0, 28000.0, 46.0],
       [1.0, 0.0, 25000.0, 34.0],
       [1.0, 0.0, 117000.0, 32.0],
       [1.0, 0.0, 76000.0, 59.0],
       [0.0, 1.0, 75000.0, 35.0],
       [0.0, 1.0, 22000.0, 25.0],
       [1.0, 0.0, 80000.0, 41.0],
       [1.0, 0.0, 112000.0, 38.0],
       [1.0, 0.0, 50000.0, 38.0],
       [0.0, 1.0, 99000.0, 36.0],
       [0.0, 1.0, 75000.0, 37.0],
       [0.0, 1.0, 80000.0, 29.0],
       [1.0, 0.0, 79000.0, 30.0],
       [0.0, 1.0, 105000.0, 47.0],
       [1.0, 0.0, 58000.0, 27.0],
       [1.0, 0.0, 61000.0, 39.0],
       [0.0, 1.0, 134000.0, 39.0],
       [1.0, 0.0, 72000.0, 41.0],
       [0.0, 1.0, 42000.0, 39.0],
       [1.0, 0.0, 133000.0, 43.0],
       [1.0, 0.0, 142000.0, 40.0],
       [0.0, 1.0, 53000.0, 35.0],
       [1.0, 0.0, 82000.0, 53.0],
       [0.0, 1.0, 30000.0, 26.0],
       [1.0, 0.0, 45000.0, 45.0],
       [1.0, 0.0, 31000.0, 27.0],
       [0.0, 1.0, 55000.0, 24.0],
       [

In [65]:
modeloRLog=linear_model.LogisticRegression(max_iter= 10000,penalty='none',fit_intercept=True, random_state=123)
modeloRLog.fit(X_train_dummy, Y_train)  
Y_pred=modeloRLog.predict(X_test_dummy)
acc_rlog=accuracy_score(Y_test, Y_pred)

print("Evaluación modelo con Regresión Logística")
print("Accuracy--> ", round(acc_rlog,3))

Evaluación modelo con Regresión Logística
Accuracy-->  0.648


In [67]:
modeloRLog2= linear_model.LogisticRegression(max_iter= 20000, penalty="none", fit_intercept=True, random_state=123)
modeloRLog2.fit(X_train_dummy,Y_train)
y_val_pred_S= modeloRLog2.predict(X_test_dummy)

In [70]:
AC_RLog_S=accuracy_score(Y_test, y_val_pred_S)
F1_RLog_S=f1_score(y_val_test,y_val_pred_S, average='macro')

print("Evaluación modelo de Regresión Logística en el Validation S")
print("Accuracy: ", round(AC_RLog_S,3))
print("F1: ", round(F1_RLog_S,3))

NameError: name 'f1_score' is not defined